In [1]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

In [2]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"

src_dbname = "classicmodels"
dst_dbname = "retail_sales_dw"

In [3]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [4]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

#connection.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
#connection.execute(f"CREATE DATABASE `{dst_dbname}`;")
connection.execute(f"USE {dst_dbname};")

connection.close()

In [5]:
sql_products = "SELECT * FROM classicmodels.products;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_products)
df_products.head(2)

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.7
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.3


In [6]:
df_products.drop('productScale', axis=1, inplace=True)
df_products.drop('productVendor', axis=1, inplace=True)
df_products.drop('productDescription', axis=1, inplace=True)
df_products.drop('MSRP', axis=1, inplace=True)

df_products.rename(columns={"productCode":"product_id"}, inplace=True)
df_products.rename(columns={"productName":"product_name"}, inplace=True)
df_products.rename(columns={"productLine":"product_line"}, inplace=True)
df_products.rename(columns={"quantityInStock":"quantity_in_stock"}, inplace=True)
df_products.rename(columns={"buyPrice":"product_price"}, inplace=True)

df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))

df_products.head(2)

,product_key,product_id,product_name,product_line,quantity_in_stock,product_price
0,1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,7933,48.81
1,2,S10_1949,1952 Alpine Renault 1300,Classic Cars,7305,98.58


In [7]:
db_operation = "insert"

tables = [('product_dim', df_products, 'product_key')]

In [8]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)